In [22]:
# import plotly px
import plotly.express as px
import pandas as pd
import plotly.graph_objects as go
import plotly.offline as py
import numpy as np

In [23]:
# import spark and create spark session with name flights
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('flights').getOrCreate()

![](imgs/Flight-cycle-of-an-aircraft.png)

In [24]:
# load a csv file in a dataframe infer schema
df = spark.read.csv('../data/On_Time_Reporting_Carrier_On_Time_Performance_(1987_present)_2013_12.csv', inferSchema=True, header=True)


In [27]:
# get the rows with Flights equals to 2
df.filter(df.Flights == 1).show(10)

22/12/21 11:10:16 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: Year, Quarter, Month, DayofMonth, DayOfWeek, FlightDate, Reporting_Airline, DOT_ID_Reporting_Airline, IATA_CODE_Reporting_Airline, Tail_Number, Flight_Number_Reporting_Airline, OriginAirportID, OriginAirportSeqID, OriginCityMarketID, Origin, OriginCityName, OriginState, OriginStateFips, OriginStateName, OriginWac, DestAirportID, DestAirportSeqID, DestCityMarketID, Dest, DestCityName, DestState, DestStateFips, DestStateName, DestWac, CRSDepTime, DepTime, DepDelay, DepDelayMinutes, DepDel15, DepartureDelayGroups, DepTimeBlk, TaxiOut, WheelsOff, WheelsOn, TaxiIn, CRSArrTime, ArrTime, ArrDelay, ArrDelayMinutes, ArrDel15, ArrivalDelayGroups, ArrTimeBlk, Cancelled, CancellationCode, Diverted, CRSElapsedTime, ActualElapsedTime, AirTime, Flights, Distance, DistanceGroup, CarrierDelay, WeatherDelay, NASDelay, SecurityDelay, LateAircraftDelay, FirstDepTime, TotalAddGTime, LongestAddGTime, DivAirportLandi

In [4]:
# import sql functions
from pyspark.sql.functions import *


In [9]:
# print df schema
df.printSchema()

root
 |-- Year: integer (nullable = true)
 |-- Quarter: integer (nullable = true)
 |-- Month: integer (nullable = true)
 |-- DayofMonth: integer (nullable = true)
 |-- DayOfWeek: integer (nullable = true)
 |-- FlightDate: timestamp (nullable = true)
 |-- Reporting_Airline: string (nullable = true)
 |-- DOT_ID_Reporting_Airline: integer (nullable = true)
 |-- IATA_CODE_Reporting_Airline: string (nullable = true)
 |-- Tail_Number: string (nullable = true)
 |-- Flight_Number_Reporting_Airline: integer (nullable = true)
 |-- OriginAirportID: integer (nullable = true)
 |-- OriginAirportSeqID: integer (nullable = true)
 |-- OriginCityMarketID: integer (nullable = true)
 |-- Origin: string (nullable = true)
 |-- OriginCityName: string (nullable = true)
 |-- OriginState: string (nullable = true)
 |-- OriginStateFips: integer (nullable = true)
 |-- OriginStateName: string (nullable = true)
 |-- OriginWac: integer (nullable = true)
 |-- DestAirportID: integer (nullable = true)
 |-- DestAirportSe

To drop:
- IATA_CODE_Reporting_Airline
- 

Meaning:
- Tail_Number: plane identifier

In [11]:
# import the data from csv in pandas dataframe
df_pd = pd.read_csv('../data/On_Time_Reporting_Carrier_On_Time_Performance_(1987_present)_2013_12.csv')

/var/folders/tx/5_6_zg1d0fd4zjwzxg1ztrth0000gn/T/ipykernel_56760/2719218514.py:2: DtypeWarning:

Columns (77,84,85) have mixed types. Specify dtype option on import or set low_memory=False.



In [14]:
# count rows in pandas dataframe
print('Number of rows in pandas dataframe: ', len(df_pd))

Number of rows in pandas dataframe:  516739


In [15]:
pd.set_option('display.max_rows', None)

In [16]:
# count the percentage of missing values in each column
print('Percentage of missing values in each column: ')
print(df_pd.isnull().sum()/len(df_pd)*100)


Percentage of missing values in each column: 
Year                                 0.000000
Quarter                              0.000000
Month                                0.000000
DayofMonth                           0.000000
DayOfWeek                            0.000000
FlightDate                           0.000000
Reporting_Airline                    0.000000
DOT_ID_Reporting_Airline             0.000000
IATA_CODE_Reporting_Airline          0.000000
Tail_Number                          0.457291
Flight_Number_Reporting_Airline      0.000000
OriginAirportID                      0.000000
OriginAirportSeqID                   0.000000
OriginCityMarketID                   0.000000
Origin                               0.000000
OriginCityName                       0.000000
OriginState                          0.000000
OriginStateFips                      0.000000
OriginStateName                      0.000000
OriginWac                            0.000000
DestAirportID                     

In [19]:
# load airport.csv as pandas dataframe
airports = pd.read_csv('airports.csv')

In [ ]:
# make a join between df_sampled and airports
df_sampled = df_sampled.merge(airports, left_on='Origin', right_on='IATA')
df_sampled = df_sampled.merge(airports, left_on='Dest', right_on='IATA')

In [ ]:
# save df_sampled as csv
df_sampled.to_csv('df_sampled.csv')

In [ ]:
df_sampled=df.toPandas()

In [17]:
# sample 100_000 rows from df_sampled
df_sampled = df_pd.sample(100)

In [ ]:
# numbers of null values in each column, show full output
pd.set_option('display.max_rows', None)
df_sampled.isnull().sum()

In [20]:
airportspy = [ dict(
        type = 'scattergeo',
        locationmode = 'USA-states',
        lon = airports['LONGITUDE'],
        lat = airports['LATITUDE'],
        text = '<b>' + airports['AIRPORT'].map(str) + '</b>',
        hoverinfo = 'text',
        marker = dict(
            size=2,
            sizemin=2,
            cmin = 10.565915336317195,
            cmax = 38.383341806128094,
            colorscale = [[0,'rgb(255, 255, 0)'],[1,'rgb(255, 0, 0)']],
            colorbar=go.ColorBar(
                 title='Score'
            ),
            opacity = 1,
            line = dict(
                width=0,
                color='rgba(68, 68, 68, 0)'
            )          
        ))]

/opt/homebrew/lib/python3.10/site-packages/plotly/graph_objs/_deprecations.py:173: DeprecationWarning:

plotly.graph_objs.ColorBar is deprecated.
Please replace it with one of the following more specific types
  - plotly.graph_objs.scatter.marker.ColorBar
  - plotly.graph_objs.surface.ColorBar
  - etc.




In [ ]:
flight_paths = []
for i in range(len(df_pd)):
      
     
    
    flight_paths.append(
        dict(
            type = 'scattergeo',
            locationmode = 'USA-states',
            lon = [ float(aux_origin['LONGITUDE_x']), float(aux_destin['LONGITUDE_y']) ],
            lat = [ float(aux_origin['LATITUDE_x']), float(aux_destin['LATITUDE_y']) ],
            mode = 'lines',
            hoverinfo = 'skip',
            line = dict(
                width = 0.5,
                #color = color[i]
            ),
            #opacity = float(df_pd.loc[i,'COUNT']/maxcount_f)*2
        )
    )
    

In [21]:



    
layout = go.Layout(
        showlegend = False, 
        geo = dict(
            #scope='north america',
            projection=dict( type='orthographic' , scale = 1.8),
            showland = True,
            showocean = True,
            showcoastlines = True,
            showcountries = True,
            landcolor = 'rgb(49, 49, 49)',
            countrycolor = 'rgb(90, 90, 90)',
            coastlinecolor = 'rgb(90, 90, 90)',
            oceancolor = 'rgb(29, 29, 29)',
            bgcolor = 'rgb(29, 29, 29)'
            #,center = dict(lon='-100', lat='36')
        ),
        margin=go.Margin(
         l=0,
         r=0,
         b=0,
         t=0,
         pad=0
        ),
        autosize=True,
        paper_bgcolor = 'rgb(29, 29, 29)',
        plot_bgcolor = 'rgb(29, 29, 29)'
                            )

fig = dict( data=flight_paths+airportspy , layout=layout )
py.iplot(fig, filename='routes-graph')

/opt/homebrew/lib/python3.10/site-packages/plotly/graph_objs/_deprecations.py:405: DeprecationWarning:

plotly.graph_objs.Margin is deprecated.
Please replace it with one of the following more specific types
  - plotly.graph_objs.layout.Margin




NameError: name 'flight_paths' is not defined